In [68]:
#Dependencies:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests
from scipy.stats import linregress
import time
import json
import os
from pathlib import Path
import hvplot.pandas

#Files to load
airline_data_to_load = Path("Resources/airlines.csv")
airport_data_to_load = Path("Resources/airports.csv")
flights_data_to_load = Path("Resources/flights200501.csv")

#Read the data and store it in a Pandas dataframe
airline_data = pd.read_csv(airline_data_to_load)
airport_data = pd.read_csv(airport_data_to_load)
flights_data = pd.read_csv(flights_data_to_load)

#Clean the data to have same column names
airline_data.rename(columns={"AIRLINE": "AIRLINE_NAME"}, inplace=True)
airline_data.rename(columns={"IATA_CODE": "AIRLINE"}, inplace=True)
airport_data.rename(columns={"IATA_CODE": "ORIGIN_AIRPORT"}, inplace=True)

#Combine the data into a single dataframe
flight_data_complete = pd.merge(flights_data, airline_data, on="AIRLINE", how="left")
flight_data_complete = pd.merge(flight_data_complete, airport_data, on="ORIGIN_AIRPORT", how="left")
flight_data_complete

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,AIRLINE_NAME,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,5,...,NaN,NaN,NaN,Alaska Airlines Inc.,Ted Stevens Anchorage International Airport,Anchorage,AK,USA,61.17432,-149.99619
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,10,...,NaN,NaN,NaN,American Airlines Inc.,Los Angeles International Airport,Los Angeles,CA,USA,33.94254,-118.40807
2,2015,1,1,4,US,840,N171US,SFO,CLT,20,...,NaN,NaN,NaN,US Airways Inc.,San Francisco International Airport,San Francisco,CA,USA,37.61900,-122.37484
3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,20,...,NaN,NaN,NaN,American Airlines Inc.,Los Angeles International Airport,Los Angeles,CA,USA,33.94254,-118.40807
4,2015,1,1,4,AS,135,N527AS,SEA,ANC,25,...,NaN,NaN,NaN,Alaska Airlines Inc.,Seattle-Tacoma International Airport,Seattle,WA,USA,47.44898,-122.30931
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
469963,2015,1,31,6,B6,839,N658JB,JFK,BQN,2359,...,NaN,NaN,NaN,JetBlue Airways,John F. Kennedy International Airport (New Yor...,New York,NY,USA,40.63975,-73.77893
469964,2015,1,31,6,DL,1887,N855NW,SEA,DTW,2359,...,NaN,NaN,NaN,Delta Air Lines Inc.,Seattle-Tacoma International Airport,Seattle,WA,USA,47.44898,-122.30931
469965,2015,1,31,6,F9,300,N218FR,DEN,TPA,2359,...,2.0,0.0,0.0,Frontier Airlines Inc.,Denver International Airport,Denver,CO,USA,39.85841,-104.66700
469966,2015,1,31,6,F9,422,N954FR,DEN,ATL,2359,...,0.0,0.0,0.0,Frontier Airlines Inc.,Denver International Airport,Denver,CO,USA,39.85841,-104.66700


In [69]:
#Clean the data and fill the missing values with 0
flight_data_cleaned=flight_data_complete.fillna("0")
flight_data_cleaned

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,AIRLINE_NAME,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,5,...,0,0,0,Alaska Airlines Inc.,Ted Stevens Anchorage International Airport,Anchorage,AK,USA,61.17432,-149.99619
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,10,...,0,0,0,American Airlines Inc.,Los Angeles International Airport,Los Angeles,CA,USA,33.94254,-118.40807
2,2015,1,1,4,US,840,N171US,SFO,CLT,20,...,0,0,0,US Airways Inc.,San Francisco International Airport,San Francisco,CA,USA,37.619,-122.37484
3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,20,...,0,0,0,American Airlines Inc.,Los Angeles International Airport,Los Angeles,CA,USA,33.94254,-118.40807
4,2015,1,1,4,AS,135,N527AS,SEA,ANC,25,...,0,0,0,Alaska Airlines Inc.,Seattle-Tacoma International Airport,Seattle,WA,USA,47.44898,-122.30931
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
469963,2015,1,31,6,B6,839,N658JB,JFK,BQN,2359,...,0,0,0,JetBlue Airways,John F. Kennedy International Airport (New Yor...,New York,NY,USA,40.63975,-73.77893
469964,2015,1,31,6,DL,1887,N855NW,SEA,DTW,2359,...,0,0,0,Delta Air Lines Inc.,Seattle-Tacoma International Airport,Seattle,WA,USA,47.44898,-122.30931
469965,2015,1,31,6,F9,300,N218FR,DEN,TPA,2359,...,2.0,0.0,0.0,Frontier Airlines Inc.,Denver International Airport,Denver,CO,USA,39.85841,-104.667
469966,2015,1,31,6,F9,422,N954FR,DEN,ATL,2359,...,0.0,0.0,0.0,Frontier Airlines Inc.,Denver International Airport,Denver,CO,USA,39.85841,-104.667


In [70]:
# convert data to numeric values

flight_data_cleaned['AIRLINE_DELAY'] = pd.to_numeric(flight_data_cleaned['AIRLINE_DELAY'], errors='coerce')
flight_data_cleaned['AIR_SYSTEM_DELAY'] = pd.to_numeric(flight_data_cleaned['AIR_SYSTEM_DELAY'], errors='coerce')
flight_data_cleaned['SECURITY_DELAY'] = pd.to_numeric(flight_data_cleaned['SECURITY_DELAY'], errors='coerce')
flight_data_cleaned['LATE_AIRCRAFT_DELAY'] = pd.to_numeric(flight_data_cleaned['LATE_AIRCRAFT_DELAY'], errors='coerce')
flight_data_cleaned['WEATHER_DELAY'] = pd.to_numeric(flight_data_cleaned['WEATHER_DELAY'], errors='coerce')

In [71]:
# crete new df where delay is > 0 
flights_all_delayed_df= flight_data_cleaned[(flight_data_cleaned['AIRLINE_DELAY'] > 1) |
                  (flight_data_cleaned['AIR_SYSTEM_DELAY'] > 1) |
                  (flight_data_cleaned['SECURITY_DELAY'] > 1) |
                  (flight_data_cleaned['LATE_AIRCRAFT_DELAY'] > 1) |
                  (flight_data_cleaned['WEATHER_DELAY'] > 1)]
flights_all_delayed_df

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,AIRLINE_NAME,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE
27,2015,1,1,4,NK,597,N528NK,MSP,FLL,115,...,0.0,0.0,0.0,Spirit Air Lines,Minneapolis-Saint Paul International Airport,Minneapolis,MN,USA,44.88055,-93.21692
30,2015,1,1,4,NK,168,N629NK,PHX,ORD,125,...,0.0,0.0,0.0,Spirit Air Lines,Phoenix Sky Harbor International Airport,Phoenix,AZ,USA,33.43417,-112.00806
35,2015,1,1,4,HA,17,N389HA,LAS,HNL,145,...,15.0,0.0,0.0,Hawaiian Airlines Inc.,McCarran International Airport,Las Vegas,NV,USA,36.08036,-115.15233
50,2015,1,1,4,B6,1030,N239JB,BQN,MCO,307,...,0.0,0.0,0.0,JetBlue Airways,Rafael Hernández Airport,Aguadilla,PR,USA,18.49486,-67.12944
52,2015,1,1,4,B6,2134,N307JB,SJU,MCO,400,...,85.0,0.0,0.0,JetBlue Airways,Luis Muñoz Marín International Airport,San Juan,PR,USA,18.43942,-66.00183
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
469945,2015,1,31,6,AS,121,N763AS,SEA,ANC,2350,...,0.0,78.0,0.0,Alaska Airlines Inc.,Seattle-Tacoma International Airport,Seattle,WA,USA,47.44898,-122.30931
469953,2015,1,31,6,B6,778,N594JB,LAS,BOS,2355,...,0.0,28.0,0.0,JetBlue Airways,McCarran International Airport,Las Vegas,NV,USA,36.08036,-115.15233
469959,2015,1,31,6,B6,98,N535JB,DEN,JFK,2357,...,0.0,11.0,13.0,JetBlue Airways,Denver International Airport,Denver,CO,USA,39.85841,-104.667
469965,2015,1,31,6,F9,300,N218FR,DEN,TPA,2359,...,2.0,0.0,0.0,Frontier Airlines Inc.,Denver International Airport,Denver,CO,USA,39.85841,-104.667


In [72]:
#What is the average delay time for each airport for airline delay for airline

#Find the average delay time for each airport delay using groupby 
average_airline_delay= flights_all_delayed_df.groupby(["AIRPORT"])["AIRLINE_DELAY"].mean()


In [73]:
#What is the average delay time for each airport for airline delay for air system

#Find the average delay time for each airport delay using groupby 
average_air_system_delay= flights_all_delayed_df.groupby(["AIRPORT"])['AIR_SYSTEM_DELAY'].mean()


In [74]:
#What is the average delay time for each airport for security

#Find the average delay time for each airline using groupby 
average_security_delay= flights_all_delayed_df.groupby(["AIRPORT"])["SECURITY_DELAY"].mean()


In [75]:
#What is the average delay time for each airport for late

#Find the average delay time for each airport using groupby 
average_late_airline_delay= flights_all_delayed_df.groupby(["AIRPORT"])["LATE_AIRCRAFT_DELAY"].mean()


In [76]:
#What is the average delay time for each airline for weather

#Find the average delay time for each airline using groupby 
average_weather_delay= flights_all_delayed_df.groupby(["AIRPORT"])["WEATHER_DELAY"].mean()


In [77]:
flights_delay_averages = pd.DataFrame({
                                    "Airline Delay AVG": average_airline_delay,
                                    "Air System Delay AVG": average_air_system_delay,
                                    "Security Delay AVG": average_security_delay,
                                    "Late Aircraft Delay AVG": average_late_airline_delay,
                                    "Weather Delay AVG": average_weather_delay 
})


In [78]:
# Create a df for the sum of the delays by each individual airport

delay_sums = flights_delay_averages.groupby('AIRPORT')[['Airline Delay AVG', 'Air System Delay AVG', 'Security Delay AVG', 'Late Aircraft Delay AVG', 'Weather Delay AVG']].sum()
delay_sums['MOST TOTAL DELAYS (minutes)'] = delay_sums[['Airline Delay AVG', 'Air System Delay AVG', 'Security Delay AVG', 'Late Aircraft Delay AVG', 'Weather Delay AVG']].sum(axis=1)
delay_sums_df = pd.DataFrame(delay_sums ['MOST TOTAL DELAYS (minutes)'])
delay_sums_df

,MOST TOTAL DELAYS (minutes)
AIRPORT,
Aberdeen Regional Airport,186.888889
Abilene Regional Airport,58.062500
Abraham Lincoln Capital Airport,56.733333
Adak Airport,27.000000
Akron-Canton Regional Airport,51.814433
...,...
Yakutat Airport,51.428571
Yampa Valley Airport (Yampa Valley Regional),66.725000
Yeager Airport,73.727273


In [79]:
# crete new df where delay is > 0 
flights_least_delayed_df= flight_data_cleaned[(flight_data_cleaned['AIRLINE_DELAY'] < 65.117605) |
                  (flight_data_cleaned['AIR_SYSTEM_DELAY'] < 65.117605) |
                  (flight_data_cleaned['SECURITY_DELAY'] < 65.117605) |
                  (flight_data_cleaned['LATE_AIRCRAFT_DELAY'] < 65.117605) |
                  (flight_data_cleaned['WEATHER_DELAY'] < 65.117605)]
flights_least_delayed_df

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,AIRLINE_NAME,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,5,...,0.0,0.0,0.0,Alaska Airlines Inc.,Ted Stevens Anchorage International Airport,Anchorage,AK,USA,61.17432,-149.99619
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,10,...,0.0,0.0,0.0,American Airlines Inc.,Los Angeles International Airport,Los Angeles,CA,USA,33.94254,-118.40807
2,2015,1,1,4,US,840,N171US,SFO,CLT,20,...,0.0,0.0,0.0,US Airways Inc.,San Francisco International Airport,San Francisco,CA,USA,37.619,-122.37484
3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,20,...,0.0,0.0,0.0,American Airlines Inc.,Los Angeles International Airport,Los Angeles,CA,USA,33.94254,-118.40807
4,2015,1,1,4,AS,135,N527AS,SEA,ANC,25,...,0.0,0.0,0.0,Alaska Airlines Inc.,Seattle-Tacoma International Airport,Seattle,WA,USA,47.44898,-122.30931
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
469963,2015,1,31,6,B6,839,N658JB,JFK,BQN,2359,...,0.0,0.0,0.0,JetBlue Airways,John F. Kennedy International Airport (New Yor...,New York,NY,USA,40.63975,-73.77893
469964,2015,1,31,6,DL,1887,N855NW,SEA,DTW,2359,...,0.0,0.0,0.0,Delta Air Lines Inc.,Seattle-Tacoma International Airport,Seattle,WA,USA,47.44898,-122.30931
469965,2015,1,31,6,F9,300,N218FR,DEN,TPA,2359,...,2.0,0.0,0.0,Frontier Airlines Inc.,Denver International Airport,Denver,CO,USA,39.85841,-104.667
469966,2015,1,31,6,F9,422,N954FR,DEN,ATL,2359,...,0.0,0.0,0.0,Frontier Airlines Inc.,Denver International Airport,Denver,CO,USA,39.85841,-104.667


In [80]:
#What is the average delay time for each airport for airline delay for airline

#Find the average delay time for each airport delay using groupby 
average_airline_delay_least= flights_least_delayed_df.groupby(["AIRPORT"])["AIRLINE_DELAY"].mean()

In [81]:
#What is the average delay time for each airport for airline delay for air system

#Find the average delay time for each airport delay using groupby 
average_air_system_delay_least= flights_least_delayed_df.groupby(["AIRPORT"])['AIR_SYSTEM_DELAY'].mean()

In [82]:
#What is the average delay time for each airport for security

#Find the average delay time for each airline using groupby 
average_security_delay_least= flights_least_delayed_df.groupby(["AIRPORT"])["SECURITY_DELAY"].mean()

In [83]:
#What is the average delay time for each airport for late

#Find the average delay time for each airport using groupby 
average_late_airline_delay_least= flights_least_delayed_df.groupby(["AIRPORT"])["LATE_AIRCRAFT_DELAY"].mean()

In [84]:
#What is the average delay time for each airline for weather

#Find the average delay time for each airline using groupby 
average_weather_delay_least= flights_least_delayed_df.groupby(["AIRPORT"])["WEATHER_DELAY"].mean()

In [85]:
flights_delay_least_averages = pd.DataFrame({
                                    "Airline Delay AVG": average_airline_delay_least,
                                    "Air System Delay AVG": average_air_system_delay_least,
                                    "Security Delay AVG": average_security_delay_least,
                                    "Late Aircraft Delay AVG": average_late_airline_delay_least,
                                    "Weather Delay AVG": average_weather_delay_least
})

In [86]:
# Create a df for the sum of the delays by each individual airport

least_delay_sums = flights_delay_least_averages.groupby('AIRPORT')[['Airline Delay AVG', 'Air System Delay AVG', 'Security Delay AVG', 'Late Aircraft Delay AVG', 'Weather Delay AVG']].sum()
least_delay_sums['LEAST TOTAL DELAYS (minutes)'] = least_delay_sums[['Airline Delay AVG', 'Air System Delay AVG', 'Security Delay AVG', 'Late Aircraft Delay AVG', 'Weather Delay AVG']].sum(axis=1)
least_delay_sums_df = pd.DataFrame(least_delay_sums ['LEAST TOTAL DELAYS (minutes)'])
least_delay_sums_df

,LEAST TOTAL DELAYS (minutes)
AIRPORT,
Aberdeen Regional Airport,27.129032
Abilene Regional Airport,11.661088
Abraham Lincoln Capital Airport,13.193798
Adak Airport,6.000000
Akron-Canton Regional Airport,8.991055
...,...
Yakutat Airport,6.000000
Yampa Valley Airport (Yampa Valley Regional),17.793333
Yeager Airport,18.431818


In [87]:
# Top 5 airports with least delays
least_delay_sums_df.sort_values(by = "LEAST TOTAL DELAYS (minutes)", axis=0,ascending=True).head(5)

,LEAST TOTAL DELAYS (minutes)
AIRPORT,
Bert Mooney Airport,0.000000
Canyonlands Field,0.000000
Lewiston-Nez Perce County Airport,0.320755
Magic Valley Regional Airport (Joslin Field),0.951613
Elko Regional Airport,1.524590


In [88]:
# top 5 airports with most delays
delay_sums_df.sort_values(by = "MOST TOTAL DELAYS (minutes)", axis=0,ascending=False).head(5)

,MOST TOTAL DELAYS (minutes)
AIRPORT,
Pocatello Regional Airport,213.555556
Aberdeen Regional Airport,186.888889
Delta County Airport,164.000000
Valdez Airport,138.000000
Santa Maria Public Airport (Capt G. Allan Hancock Field),126.285714
